# FEATURE EXTRACTION - DEMO

In [ ]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#project variables
demopath=os.getcwd()


In [ ]:
#define features path
dreamspath=demopath+"/DREAMS"
featurespath=dreamspath+"/features"
if not os.path.isdir(featurespath):
    os.mkdir(featurespath)
        
windowDurations=[0.5,1,1.5,2]

## define a fixed samplerate

In [ ]:
samplerate=200

## Load data

In [ ]:
signals, annotations, signalsMetadata = loadDREAMSSpindlesDemo(demopath)

In [ ]:
signalsMetadata

In [ ]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)

In [ ]:
annotations.head()

## Preprocess

broadband in 0-40Hz

In [ ]:
def preprocessVector(vector,samplerate):
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Z-score
    m=np.mean(vector)
    s=np.std(vector)
    vector=(vector-m)/s
    return vector

In [ ]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

## Extract features

### folder estructure

In [ ]:
fspath=featurespath+'/'+str(samplerate)+'fs/'
if not os.path.isdir(fspath):
    os.mkdir(fspath)
for window in windowDurations:
    windowPath=fspath+str(window)+'win'
    if not os.path.isdir(windowPath):
        os.mkdir(windowPath)
    for subject in signalsMetadata.subjectId:
        subjectPath=windowPath+'/'+subject
        if not os.path.isdir(subjectPath):
            os.mkdir(subjectPath)

### band definition

In [ ]:
bands={
    'delta1':[0.1,2],
    'delta2':[2,4],
    'theta':[4,8],
    'alpha':[8,13],
    'sigma':[11,16],
    'beta1':[13,19],
    'beta2':[19,30]
    }

### computation

In [ ]:
for window in windowDurations:
    windowPath=featurespath+'/'+str(samplerate)+'fs/'+str(window)
    for ind, row in signalsMetadata.iterrows():
        subject=row.subjectId
        subjectPath=windowPath+'/'+subject
        signal=signals[subject]
        #need to define the time vector for each signal:
        timepoints=np.arange(len(signal))/samplerate
        
        #6. Hjort Activity
        characteristic='hjortActivity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortActivity(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortActivity(filtered,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #7. Hjort Mobility
        characteristic='hjortMobility'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortMobility(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortMobility(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #8. Hjort Complexity
        characteristic='hjortComplexity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortComplexity(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortComplexity(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #9. PetrosianFractalDimension
        characteristic='petrosian'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=petrosianFractalDimension(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=petrosianFractalDimension(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        
        #12. sigma index
        characteristic='sigmaIndex'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=sigmaindex(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
            